In [1]:
from urllib.request import urlopen   
from urllib.error import URLError, HTTPError
import pandas as pd
import os
import matplotlib.pyplot as plt
import sqlite3
import numpy as np
import seaborn as sns; sns.set(style="ticks", color_codes=True)
from sklearn.model_selection import train_test_split

pd.set_option('display.float_format', lambda x: '%.3f' % x) # do not display exponents
%matplotlib inline
pd.set_option('display.max_columns', None)

In [2]:
movie_agg_df = pd.read_csv('imdb.agg.tsv', sep='\t', quotechar='"').set_index('tconst')
movie_agg_df.head(3)

,originalTitle,isAdult,startYear,runtimeMinutes,averageRating,numVotes,AR,AU,BR,CA,DE,DK,ES,FI,FR,GB,GR,HU,IN,IT,JP,MX,PL,PT,RU,SE,TR,US,XWG,XWW,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,actor_1,actor_2,director_1,director_2,writer_1,writer_2,weightedAvg_actor_1_attr,weightedAvg_actor_2_attr,weightedAvg_director_1_attr,weightedAvg_director_2_attr,weightedAvg_writer_1_attr,weightedAvg_writer_2_attr,Numvotes_actor_1_attr,Numvotes_actor_2_attr,Numvotes_director_1_attr,Numvotes_director_2_attr,Numvotes_writer_1_attr,Numvotes_writer_2_attr
tconst,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
tt0002423,Madame DuBarry,0,1919,85.000,6.900,693.000,0,0,1,0,1,1,0,1,1,0,0,1,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nm0417837,nm0509573,nm0523932,NaN,nm0266183,nm0473134,5.840,5.910,6.430,6.480,6.200,6.420,1011.000,1253.000,2381.000,1963.000,242.000,2048.000
tt0002445,Quo Vadis?,0,1913,120.000,6.200,235.000,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nm0637019,nm0784988,nm0345500,NaN,nm0797197,NaN,7.200,7.200,5.800,5.800,5.600,5.600,266.000,266.000,183.000,183.000,206.000,206.000
tt0002628,Der Andere,0,1913,48.000,5.700,62.000,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,nm0060168,nm0361518,nm0533048,NaN,nm0511317,nm1857018,7.200,7.200,5.800,5.800,5.600,5.600,266.000,266.000,183.000,183.000,206.000,206.000


In [3]:
# exclude cols not needed for modeling
modeling_colnames_exclude = ['originalTitle','actor_1','actor_2','director_1','director_2','writer_1','writer_2']
modeling_colnames = list(set(movie_agg_df.columns) - set(modeling_colnames_exclude))
modeling_df = movie_agg_df[modeling_colnames]
modeling_df.head()

,Adventure,XWW,weightedAvg_director_1_attr,US,Documentary,Numvotes_actor_2_attr,RU,weightedAvg_director_2_attr,weightedAvg_actor_1_attr,PT,JP,Mystery,Comedy,War,BR,Action,Animation,weightedAvg_writer_2_attr,Numvotes_actor_1_attr,HU,TR,Sci-Fi,Adult,startYear,MX,AR,Horror,ES,Talk-Show,News,Music,Numvotes_writer_2_attr,Reality-TV,Biography,SE,Romance,Sport,Musical,History,Game-Show,FI,isAdult,GB,Short,Family,IN,PL,CA,Crime,GR,Numvotes_director_1_attr,XWG,Numvotes_writer_1_attr,Numvotes_director_2_attr,numVotes,AU,DK,Film-Noir,Drama,weightedAvg_writer_1_attr,runtimeMinutes,Fantasy,weightedAvg_actor_2_attr,averageRating,DE,Thriller,Western,IT,FR
tconst,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
tt0002423,0,0,6.430,1,0,1253.000,1,6.480,5.840,0,0,0,0,0,1,0,0,6.420,1011.000,1,0,0,0,1919,0,0,0,0,0,0,0,2048.000,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,2381.000,0,242.000,1963.000,693.000,0,1,0,1,6.200,85.000,0,5.910,6.900,1,0,0,0,1
tt0002445,0,0,5.800,0,0,266.000,1,5.800,7.200,0,0,0,0,0,0,0,0,5.600,266.000,1,0,0,0,1913,0,0,0,0,0,0,0,206.000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,183.000,0,206.000,183.000,235.000,0,1,0,1,5.600,120.000,0,7.200,6.200,0,0,0,1,0
tt0002628,0,0,5.800,0,0,266.000,0,5.800,7.200,1,0,0,0,0,0,0,0,5.600,266.000,1,0,0,0,1913,0,0,0,0,0,0,0,206.000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,183.000,0,206.000,183.000,62.000,0,1,0,1,5.600,48.000,0,7.200,5.700,1,1,0,0,0
tt0002646,0,0,5.480,0,0,266.000,1,5.480,7.200,0,0,0,0,0,0,0,0,5.600,266.000,1,0,0,0,1913,0,0,0,0,0,0,0,206.000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,156.000,0,206.000,156.000,304.000,0,1,0,1,5.600,121.000,0,7.200,6.800,1,0,0,0,0
tt0002767,0,0,5.800,1,0,266.000,0,5.800,7.200,0,0,0,0,0,0,0,0,5.600,266.000,1,0,0,0,1913,0,0,0,0,0,0,0,206.000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,183.000,0,206.000,183.000,35.000,0,0,0,1,5.600,69.000,0,7.200,5.400,0,0,0,0,0


In [4]:
modeling_df.describe()

,Adventure,XWW,weightedAvg_director_1_attr,US,Documentary,Numvotes_actor_2_attr,RU,weightedAvg_director_2_attr,weightedAvg_actor_1_attr,PT,JP,Mystery,Comedy,War,BR,Action,Animation,weightedAvg_writer_2_attr,Numvotes_actor_1_attr,HU,TR,Sci-Fi,Adult,startYear,MX,AR,Horror,ES,Talk-Show,News,Music,Numvotes_writer_2_attr,Reality-TV,Biography,SE,Romance,Sport,Musical,History,Game-Show,FI,isAdult,GB,Short,Family,IN,PL,CA,Crime,GR,Numvotes_director_1_attr,XWG,Numvotes_writer_1_attr,Numvotes_director_2_attr,numVotes,AU,DK,Film-Noir,Drama,weightedAvg_writer_1_attr,runtimeMinutes,Fantasy,weightedAvg_actor_2_attr,averageRating,DE,Thriller,Western,IT,FR
count,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000,136384.000
mean,0.075,0.227,6.186,0.434,0.095,29666.296,0.185,6.265,6.313,0.192,0.072,0.046,0.300,0.025,0.259,0.126,0.020,6.204,47091.411,0.210,0.119,0.032,0.007,1992.609,0.104,0.122,0.088,0.230,0.000,0.002,0.025,23939.198,0.000,0.032,0.157,0.136,0.013,0.024,0.029,0.000,0.176,0.008,0.107,0.000,0.042,0.067,0.209,0.118,0.106,0.234,14710.530,0.125,17825.011,16741.072,5556.703,0.034,0.132,0.005,0.486,6.204,97.499,0.035,6.224,6.123,0.170,0.103,0.019,0.229,0.260
std,0.263,0.419,0.747,0.496,0.294,73605.046,0.388,0.345,0.765,0.394,0.258,0.210,0.458,0.156,0.438,0.332,0.140,0.406,119207.453,0.407,0.324,0.176,0.084,24.083,0.306,0.327,0.283,0.421,0.000,0.042,0.157,31263.594,0.009,0.175,0.364,0.343,0.113,0.152,0.167,0.003,0.381,0.088,0.310,0.000,0.201,0.250,0.407,0.322,0.307,0.423,44067.589,0.331,36349.250,18376.162,39710.004,0.180,0.338,0.073,0.500,0.541,48.421,0.185,0.658,1.262,0.375,0.304,0.136,0.420,0.439
min,0.000,0.000,1.000,0.000,0.000,31.000,0.000,1.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,31.000,0.000,0.000,0.000,0.000,1913.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,31.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,31.000,0.000,31.000,31.000,31.000,0.000,0.000,0.000,0.000,1.000,7.000,0.000,1.000,1.000,0.000,0.000,0.000,0.000,0.000
25%,0.000,0.000,5.981,0.000,0.000,3436.000,0.000,6.103,6.074,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,6.038,1867.750,0.000,0.000,0.000,0.000,1978.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,6883.075,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1234.000,0.000,4673.389,4456.074,67.000,0.000,0.000,0.000,0.000,6.042,86.000,0.000,6.020,5.400,0.000,0.000,0.000,0.000,0.000
50%,0.000,0.000,6.112,0.000,0.000,17756.436,0.000,6.240,6.269,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,6.114,18752.585,0.000,0.000,0.000,0.000,2002.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,25276.672,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,10956.500,0.000,18068.073,10769.941,181.000,0.000,0.000,0.000,0.000,6.126,95.000,0.000,6.159,6.300,0.000,0.000,0.000,0.000,0.000
75%,0.000,0.000,6.450,1.000,0.000,42052.419,0.000,6.462,6.653,0.000,0.000,0.000,1.000,0.000,1.000,0.000,0.000,6.343,66394.750,0.000,0.000,0.000,0.000,2012.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,35887.165,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,19101.679,0.000,24545.478,27910.148,741.000,0.000,0

In [5]:
# noticing runtime of 14400 minutes (10 days) - this is an extreme outlier
runtime_cutoff = (modeling_df["runtimeMinutes"].quantile(0.999))
modeling_df = modeling_df[modeling_df["runtimeMinutes"] < runtime_cutoff]

In [7]:
features = modeling_df
# Labels are the values we want to predict
labels = np.array(features['averageRating'])
# Remove the labels from the features
features= features.drop('averageRating', axis = 1)
# Saving feature names for later use
feature_list = list(features.columns)
features = np.array(features)

In [8]:
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = \
train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [9]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (102183, 68)
Training Labels Shape: (102183,)
Testing Features Shape: (34062, 68)
Testing Labels Shape: (34062,)


## Baseline

In [21]:
# establish baseline error, if we were just to guess average, what would the results look like?
baseline_errors = abs(test_labels.mean() - test_labels)
print('Baseline Mean Absolute Error: ', round(np.mean(baseline_errors), 2))
mape = 100 * (baseline_errors / test_labels)
accuracy = 100 - np.mean(mape)
print('Baseline MAPE Accuracy:', round(accuracy, 2), '%.')

Baseline Mean Absolute Error:  0.99
Baseline MAPE Accuracy: 80.24 %.


In [23]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 1000, random_state = 11)
rf.fit(train_features, train_labels);

In [24]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2))

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
accuracy = 100 - np.mean(mape)
print('MAPE Accuracy:', round(accuracy, 2), '%.')

Mean Absolute Error: 0.69
MAPE Accuracy: 86.31 %.


In [25]:
# Get feature importances
importances = list(rf.feature_importances_)
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:30} Importance: {}'.format(*pair)) for pair in feature_importances[:20]];

Variable: weightedAvg_director_1_attr    Importance: 0.14
Variable: numVotes                       Importance: 0.12
Variable: runtimeMinutes                 Importance: 0.08
Variable: Horror                         Importance: 0.07
Variable: Documentary                    Importance: 0.07
Variable: weightedAvg_actor_1_attr       Importance: 0.06
Variable: Numvotes_actor_1_attr          Importance: 0.04
Variable: weightedAvg_actor_2_attr       Importance: 0.04
Variable: weightedAvg_writer_1_attr      Importance: 0.03
Variable: Drama                          Importance: 0.03
Variable: Numvotes_actor_2_attr          Importance: 0.03
Variable: startYear                      Importance: 0.03
Variable: Numvotes_director_1_attr       Importance: 0.03
Variable: weightedAvg_writer_2_attr      Importance: 0.02
Variable: Numvotes_writer_2_attr         Importance: 0.02
Variable: Numvotes_writer_1_attr         Importance: 0.02
Variable: Numvotes_director_2_attr       Importance: 0.02
Variable: weig

In [ ]:
# TODO - should remove variables with very low or 0 important and rerun

## Finding optimal params

In [10]:
n_estimators=[200,500,1000,1500,2000]
#max_features = ['auto', 'sqrt', 'log2']
max_depth = [3,5,7]
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
grid_param = {'n_estimators': n_estimators,
 #'max_features': max_features,
 'max_depth': max_depth,
 #'min_samples_split': min_samples_split,
 #'min_samples_leaf': min_samples_leaf,
 'bootstrap': bootstrap}

In [ ]:
# tried to do grid search first, computer did not have nearly enough resources. settle for randomized search
from sklearn.ensemble import RandomForestRegressor
# the model prior to hyperparameter optimization
RFR=RandomForestRegressor(random_state=1)
from sklearn.model_selection import RandomizedSearchCV
RFR_random = RandomizedSearchCV(estimator = RFR, param_distributions = grid_param, n_iter = 15, cv = 3, verbose=2, random_state=11, n_jobs = -1)
RFR_random.fit(features, labels) 
print(RFR_random.best_params_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


# Trying XGBoost

In [15]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [33]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', learning_rate = 0.1,
                max_depth = 7, n_estimators = 2000, subsample = 0.7)

In [34]:
xg_reg.fit(train_features,train_labels)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=2000,
       n_jobs=1, nthread=None, objective='reg:squarederror',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=None, subsample=0.7, verbosity=1)

In [35]:
predictions = xg_reg.predict(test_features)

In [36]:
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2))

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
accuracy = 100 - np.mean(mape)
print('MAPE Accuracy:', round(accuracy, 2), '%.')

Mean Absolute Error: 0.69
MAPE Accuracy: 86.52 %.


### Finding optimal params

In [12]:
from sklearn.model_selection import RandomizedSearchCV
import time

In [40]:
xgb_params = {
'learning_rate': [0.01, 0.1],
'n_estimators': [100,500,1000,2000],
'max_depth': [3, 5, 7],
'gamma': [0, 0.5, 1],
'subsample': [0.7, 1],
'alpha' : [0.5,2,10]
}

In [46]:
xg_reg_random_cv = RandomizedSearchCV(estimator = xg_reg, param_distributions = xgb_params, n_iter = 5, cv = 3,
                                verbose=2, random_state=11, n_jobs = -1)
xg_reg_random_cv.fit(features, labels) 
print(xg_reg_random_cv.best_params_)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  15 out of  15 | elapsed:  6.9min finished


{'subsample': 0.7, 'n_estimators': 500, 'max_depth': 7, 'learning_rate': 0.01, 'gamma': 0.5, 'alpha': 2}


In [45]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', learning_rate = 0.1,
                max_depth = 5, n_estimators = 500, subsample = 0.7, gamma = 0.5, alpha = 2)
xg_reg.fit(train_features,train_labels)
predictions = xg_reg.predict(test_features)
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2))

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
accuracy = 100 - np.mean(mape)
print('MAPE Accuracy:', round(accuracy, 2), '%.')

Mean Absolute Error: 0.67
MAPE Accuracy: 86.73 %.


Using random search with CV, we see a slight increase in accuracy with MAE going from 0.69 to 0.67